In [1]:
#importing pandas_data_reader to download data from yahoo finance
import pandas as pd
import pandas_datareader as pdr


In [2]:
df=pd.read_csv(r"C:\Users\Asus\Documents\Stock-MArket-Forecasting\AAPL.csv")

In [3]:
df.head

<bound method NDFrame.head of       Unnamed: 0 symbol                       date    close     high  \
0              0   AAPL  2015-05-27 00:00:00+00:00  132.045  132.260   
1              1   AAPL  2015-05-28 00:00:00+00:00  131.780  131.950   
2              2   AAPL  2015-05-29 00:00:00+00:00  130.280  131.450   
3              3   AAPL  2015-06-01 00:00:00+00:00  130.535  131.390   
4              4   AAPL  2015-06-02 00:00:00+00:00  129.960  130.655   
...          ...    ...                        ...      ...      ...   
1253        1253   AAPL  2020-05-18 00:00:00+00:00  314.960  316.500   
1254        1254   AAPL  2020-05-19 00:00:00+00:00  313.140  318.520   
1255        1255   AAPL  2020-05-20 00:00:00+00:00  319.230  319.520   
1256        1256   AAPL  2020-05-21 00:00:00+00:00  316.850  320.890   
1257        1257   AAPL  2020-05-22 00:00:00+00:00  318.890  319.230   

           low    open    volume    adjClose     adjHigh      adjLow  \
0     130.0500  130.34  45833246 

In [4]:
import numpy as np
df1=df.reset_index()['close']

In [5]:
df1=np.array(df1).reshape(-1,1)

In [6]:
# scaling the data not necessary for multiple perceptron
# "#from sklearn.preprocessing import MinMaxScaler
# scaler=MinMaxScaler(feature_range=(0,1))
# #reshape (-1,1)convert the given matrix into that many rows and 1 columns e.g 100 item convert into 100*1 2-d array
# df1=scaler.fit_transform(np.array(df1).reshape(-1,1))"

In [7]:
#splitting the dataset for training and test set preparation
training_size=int(len(df)*.65)
test_size=len(df)-training_size
training_size,test_size,len(df)

(817, 441, 1258)

In [8]:
df1.shape

(1258, 1)

In [9]:
df1[:,:]

array([[132.045],
       [131.78 ],
       [130.28 ],
       ...,
       [319.23 ],
       [316.85 ],
       [318.89 ]])

In [10]:
#first .65 percent for training and remaining for test set preparation
stock_train=df1[:training_size,:]
stock_test=df1[training_size:len(df1),:]

In [11]:
stock_test.shape,stock_train.shape

((441, 1), (817, 1))

In [12]:
#preparing the training and test set as we want 100 previous day data to predict the next day
class stock:
    def __init__(self,stock):
        self.stock_train=stock


    def gen_train(self,seq_len):
        self.input_train=[]
        self.output_train=[]
        for i in range((len(self.stock_train))-seq_len-1):
            x=np.array(self.stock_train[i:i+seq_len,0])
            y=np.array(self.stock_train[i+seq_len+1,0],np.float64)
            self.input_train.append(x)
            self.output_train.append(y)
        

        return np.array(self.input_train),np.array(self.output_train)
    
    


In [13]:
#getting the training and test data 
Stock_train=stock(stock_train)
X_train,Y_train=Stock_train.gen_train(100)

In [14]:
Stock_test=stock(stock_test)
X_test,Y_test=Stock_test.gen_train(100)

In [15]:
X_test.shape,X_train.shape

((340, 100), (716, 100))

In [16]:
X_train

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [ ]:
stock_train.head

In [ ]:
#importing the tensorflow and necesssary for implementation
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='relu'))
model.compile(optimizer="adam", loss="mean_squared_error",metrics = ['accuracy'])


In [ ]:
#fitting the training set and training actual value for the perceptron method to fit
model.fit(X_train, Y_train,batch_size=32,epochs=40)

In [ ]:
model.evaluate(X_test, Y_test)

In [ ]:
#prediction made by our model
test_predict=model.predict(X_test)

In [ ]:
train_predict=model.predict(X_train)

In [ ]:
train_predict.shape

In [ ]:
#plotting our result
import pandas as pd
import matplotlib.pyplot as plt
# plt.plot(scaler.inverse_transform(df1))


In [ ]:
train_predict.head

In [ ]:
# train_predict=scaler.inverse_transform(train_predict)
# test_predict=scaler.inverse_transform(test_predict)
# df1=scaler.inverse_transform(df1)

In [ ]:
#the orange line shows our prediction on training data while the greenline shows our prediction on the testset
#the blue line is the actual closing price for the data
import numpy as np
seq_len=100

Y_train_plot=np.empty_like(df1)
Y_train_plot[:,:]=np.nan
Y_train_plot[seq_len+1:len(X_train)+seq_len+1,:]=train_predict
#similarily for test
Y_test_plot=np.empty_like(df1)
Y_test_plot[:,:]=np.nan
Y_test_plot[len(X_train)+2*seq_len+1:len(df1)-1,:]=test_predict

plt.plot(df1)
plt.plot(Y_train_plot)
plt.plot(Y_test_plot)
